# **Exemplo 3 - Circuito do Teleporte**

Neste notebook, iremos aprender como ocorre o teleporte do estado de um qubit entre diferentes posições de memória.

<img src = teleporte.png>

In [1]:
# Importando os pacotes necessários
import netsquid as ns
from netsquid.components.qmemory import QuantumMemory
import netsquid.components.instructions as instr

<br> <br> Usando uma memória quântica com 3 posições de memória, nós demostramos como teleportar o estado de um qubit armazenado na posição 0 (qubit 1) para o qubit na posição 2 (qubit 3).

In [2]:
# Criando uma memória quântica com 3 qubits
qmemory3 = QuantumMemory(name="ExampleQMem3" , num_positions=3)
instr.INSTR_INIT(qmemory3, positions = [0]) # Inicializando um qubit na posição 0
instr.INSTR_INIT(qmemory3, positions = [1]) # Inicializando um qubit na posição 1
instr.INSTR_INIT(qmemory3, positions = [2]) # Inicializando um qubit na posição 2

In [3]:
qmem3_s0, = qmemory3.peek(positions=[0]) # Espiando o qubit na posição 0
qmem3_s1, = qmemory3.peek(positions=[1]) # Espiando o qubit na posição 1
qmem3_s2, = qmemory3.peek(positions=[2]) # Espiando o qubit na posição 2
print("qmem3_s0 as ket", qmem3_s0.qstate.qrepr) # Printa o estado do qubit na posição 0
print("qmem3_s1 as ket", qmem3_s1.qstate.qrepr) # Printa o estado do qubit na posição 1
print("qmem3_s2 as ket", qmem3_s2.qstate.qrepr) # Printa o estado do qubit na posição 2

qmem3_s0 as ket KetRepr(num_qubits=1,
ket=
[[1.+0.j]
 [0.+0.j]])
qmem3_s1 as ket KetRepr(num_qubits=1,
ket=
[[1.+0.j]
 [0.+0.j]])
qmem3_s2 as ket KetRepr(num_qubits=1,
ket=
[[1.+0.j]
 [0.+0.j]])


Vemos que todos os três qubits estão no estado $|0\rangle$. <br> <br>
Agora, iremos mudar o estado do qubit 1 para o estado <br> <br>
$$|0_y \rangle = \frac{1}{\sqrt 2}(|0\rangle + i |1\rangle)$$ <br> <br>
Nosso objetivo é teleportar esse estado do qubit 1 (slot 0) para o qubit 3 (slot 2).

In [4]:
instr.INSTR_H(qmemory3, positions=[0]) # Aplicando a porta Hadamard na posição 0
instr.INSTR_S(qmemory3, positions=[0]) # Aplicando a porta S na posição 0
print("qmem3_s0 as ket", qmem3_s0.qstate.qrepr) # Printa o estado do qubit na posição 0

qmem3_s0 as ket KetRepr(num_qubits=1,
ket=
[[0.70710678+0.j        ]
 [0.        +0.70710678j]])


O qubit 1 (slot 0) agora é o estado
$$|\psi\rangle = \frac{1}{\sqrt 2}(|0\rangle + i|1\rangle)$$
Para constar, esse estado também é predefinido no NetSquid. É o estado ns.y0

In [5]:
print(ns.y0)

[[0.70710678+0.j        ]
 [0.        +0.70710678j]]


Em seguida, nós emaranharemos os qubits 2 e 3, aplicando a porta H e a CNOT ( conforme a imagem)

In [6]:
instr.INSTR_H(qmemory3, positions=[1]) # Aplicando a porta Hadamard na posição 1
instr.INSTR_CNOT(qmemory3, positions=[1,2]) # Aplicando a porta CNOT com slot 1 como controle e na posição como alvo.

In [7]:
print("qmem3_s0 as ket", qmem3_s0.qstate.qrepr) # Printa o estado do qubit na posição 0
print("qmem3_s1 as ket", qmem3_s1.qstate.qrepr) # Printa o estado do qubit na posição 1
print("qmem3_s2 as ket", qmem3_s2.qstate.qrepr) # Printa o estado do qubit na posição 2

qmem3_s0 as ket KetRepr(num_qubits=1,
ket=
[[0.70710678+0.j        ]
 [0.        +0.70710678j]])
qmem3_s1 as ket KetRepr(num_qubits=2,
ket=
[[0.70710678+0.j]
 [0.        +0.j]
 [0.        +0.j]
 [0.70710678+0.j]])
qmem3_s2 as ket KetRepr(num_qubits=2,
ket=
[[0.70710678+0.j]
 [0.        +0.j]
 [0.        +0.j]
 [0.70710678+0.j]])


Vemos que o qubit 1 está no estado $|0_y\rangle$ 
Já os qubits 2 e 3 estão compartilhando um estado de Bell <br> <br>
$$|\Phi_+\rangle = \frac{1}{\sqrt 2} \begin{pmatrix}
1 \\0
 \\0
 \\1
\end{pmatrix} = \frac{1}{\sqrt 2}(|00\rangle + |11\rangle) $$ <br> <br>

Agora, nós realizamos a medição do estado de Bell ( BSM - Bell State Measurement ) nos qubits do slot 0 e slot 1 (Conforme a imagem). Para isso, aplicamos as portas CNOT e H nos dois qubits.

In [8]:
instr.INSTR_CNOT(qmemory3, positions=[0,1])
instr.INSTR_H(qmemory3, positions=[0]) # Aplicando a porta Hadamard na posição 0

<br> <br> Medindo os qubits 1 e 2

In [9]:
m1 = instr.INSTR_MEASURE(qmemory3, positions=[0]) # Medindo o qubit na posição 0 (qubit 1)
m2 = instr.INSTR_MEASURE(qmemory3, positions=[1]) # Medindo o qubit na posição 1 (qubit 2)

<br> <br> Dependendo do resultado da medição, agora realizamos operações de correção.
- Se a medição do qubit no slot 1 resultou em '1', aplicamos a porta X ao qubit no slot 2.
- Se a medição do qubit no slot 0 resultou em '1', aplicamos o Z ao qubit no slot 2.

In [10]:
if m2[0] == 1: # Medição do qubit na posição 1 resultando em "1"
    print("aplicando a porta X")
    instr.INSTR_X(qmemory3, positions=[2]) # Aplicando a porta X no qubit na posição 2
if m1[0] ==1: # Medição do qubit na posição 0 resultando em "1"
    print("aplicando a porta Z")
    instr.INSTR_Z(qmemory3, positions=[2]) # Aplicando a porta Z no qubit na posição 2

aplicando a porta Z


O qubit no slot 2 agora está no estado:


In [11]:
print("qmem3_s2 as ket", qmem3_s2.qstate.qrepr)

qmem3_s2 as ket KetRepr(num_qubits=1,
ket=
[[7.07106781e-01-1.73191211e-16j]
 [1.73191211e-16+7.07106781e-01j]])


Para comparar quão bem conseguimos teleportar o estado do qubit 1 para o qubit 3, calculamos a fidelidade (uma medida de quão semelhante é o estado teleportado ao estado original que queríamos teleportar)

<span style="color:orange"> class netsquid.qubits.qubitapi.fidelity(qubits, reference_state, squared=False)</span>

In [13]:
print("Fidelidade do estado teleportado: {}".format(
    ns.qubits.fidelity([qmem3_s2], ns.y0))) # Compara o estado teleportado com o estado que se deseja.

Fidelidade do estado teleportado: 0.9999999999999998


## **Sugestão de Prática**

- Tente teleportar outros estados;
- Em nosso exemplo nós teleportamos ns.y0;
- Tente teleportar o estado ns.h0;
- $|+\rangle = \frac{1}{\sqrt 2}(|0\rangle + |1\rangle)$;
- ou o estado ns.y1;
- $|1_y\rangle = \frac{1}{\sqrt 2}(|0\rangle - i |1\rangle)$;
- Você pode modificar o exemplo acima ou criar o seu próprio. Lembre-se de alterar o estado de destino ao calcular a fidelidade correspondente